# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,west island,-12.1568,96.8225,27.99,89,75,6.17,CC,1711935139
1,1,new norfolk,-42.7826,147.0587,19.40,49,100,1.34,AU,1711935238
2,2,whitehorse,60.7161,-135.0538,4.43,30,75,9.77,CA,1711935164
3,3,gaozhou,21.9392,110.8461,29.98,64,4,6.19,CN,1711935239
4,4,grytviken,-54.2811,-36.5092,-2.86,68,15,1.14,GS,1711935180


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points("Lng", "Lat", geo = True, tiles = "OSM", frame_width = 700, frame_height = 500,
                                      size = "Humidity", color = "City")

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = city_data_df.loc[(city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 22)]
city_data_df = city_data_df.loc[(city_data_df["Wind Speed"] < 5)]
city_data_df = city_data_df.loc[(city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
32,32,ghat,24.9647,10.1728,23.92,9,0,1.10,LY,1711935244
77,77,badr hunayn,23.7829,38.7905,25.51,49,0,1.67,SA,1711935253
154,154,salina cruz,16.1667,-95.2000,26.89,76,0,4.12,MX,1711935268
181,181,arraial do cabo,-22.9661,-42.0278,23.71,87,0,1.57,BR,1711935273
239,239,weston,26.1004,-80.3998,23.33,57,0,2.57,US,1711934610
339,339,acapulco de juarez,16.8634,-99.8901,26.90,78,0,3.60,MX,1711935307
345,345,thongwa,16.7619,96.5278,25.09,46,0,1.77,MM,1711935309
403,403,fort bragg,35.1390,-79.0060,23.91,47,0,2.57,US,1711935321
530,530,natal,-5.7950,-35.2094,25.36,94,0,1.54,BR,1711935243
532,532,conde,-7.2597,-34.9075,24.70,94,0,1.54,BR,1711935345


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
city = city_data_df["City"]
country = city_data_df["Country"]
lat = city_data_df["Lat"]
lng = city_data_df["Lng"]
hum = city_data_df["Humidity"]

hotel_df = pd.DataFrame({"City": city, "Country": country, "Lat": lat, "Lng": lng, "Humidity": hum})

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
32,ghat,LY,24.9647,10.1728,9,
77,badr hunayn,SA,23.7829,38.7905,49,
154,salina cruz,MX,16.1667,-95.2000,76,
181,arraial do cabo,BR,-22.9661,-42.0278,87,
239,weston,US,26.1004,-80.3998,57,
339,acapulco de juarez,MX,16.8634,-99.8901,78,
345,thongwa,MM,16.7619,96.5278,46,
403,fort bragg,US,35.1390,-79.0060,47,
530,natal,BR,-5.7950,-35.2094,94,
532,conde,BR,-7.2597,-34.9075,94,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
ghat - nearest hotel: فندق تاسيلي
badr hunayn - nearest hotel: No hotel found
salina cruz - nearest hotel: Hotel Mision San José
arraial do cabo - nearest hotel: No hotel found
weston - nearest hotel: TownePlace Suites by Marriott Fort Lauderdale Weston
acapulco de juarez - nearest hotel: Hotel del Valle
thongwa - nearest hotel: No hotel found
fort bragg - nearest hotel: Airborne Inn Lodging
natal - nearest hotel: Natal Palace Hotel
conde - nearest hotel: Paladino
chichen-itza - nearest hotel: Hotel Villas Arqueológicas
the hammocks - nearest hotel: TownePlace Suites Miami Kendall West


,City,Country,Lat,Lng,Humidity,Hotel Name
32,ghat,LY,24.9647,10.1728,9,فندق تاسيلي
77,badr hunayn,SA,23.7829,38.7905,49,No hotel found
154,salina cruz,MX,16.1667,-95.2000,76,Hotel Mision San José
181,arraial do cabo,BR,-22.9661,-42.0278,87,No hotel found
239,weston,US,26.1004,-80.3998,57,TownePlace Suites by Marriott Fort Lauderdale ...
339,acapulco de juarez,MX,16.8634,-99.8901,78,Hotel del Valle
345,thongwa,MM,16.7619,96.5278,46,No hotel found
403,fort bragg,US,35.1390,-79.0060,47,Airborne Inn Lodging
530,natal,BR,-5.7950,-35.2094,94,Natal Palace Hotel
532,conde,BR,-7.2597,-34.9075,94,Paladino


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
map_plot2 = hotel_df.hvplot.points("Lng", "Lat", geo = True, tiles = "OSM", frame_width = 700, frame_height = 500,
                                      size = "Humidity", color = "City", hover_cols = ["Hotel Name", "Country"])

# Display the map
map_plot2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)